## In this notebook, I analyze [this Kaggle dataset](https://www.kaggle.com/rikdifos/credit-card-approval-prediction). The dataset consists of demographic and financial data for accounts at a bank (unspecified) along with a credit history of that account. The same customer at the bank may have multiple accounts attached to them. The goal of the notebook is to clean up the data and construct a label for each customer using all of their accounts as good or bad credit. This notebook will later be used to build a model to predict the credit score of customers using their demographic and financial data, to quantify the risk of opening a credit account.

### Data Cleaning

In [311]:
import sys
import time
import pickle
import itertools
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn

Let's begin by loading our two dataframes.

In [95]:
with open("application_record.csv", "r") as app_data:
    app_df = pd.read_csv(app_data)
    

In [130]:
print(app_df)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
1       5008805           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
4       5008809           F            N               Y             0   
...         ...         ...          ...             ...           ...   
438552  6840104           M            N               Y             0   
438553  6840222           F            N               N             0   
438554  6841878           F            N               N             0   
438555  6842765           F            N               Y             0   
438556  6842885           F            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

In [97]:
with open("credit_record.csv", "r") as credit_data:
    credit_df = pd.read_csv(credit_data)

In [98]:
print(credit_df)

              ID  MONTHS_BALANCE STATUS
0        5001711               0      X
1        5001711              -1      0
2        5001711              -2      0
3        5001711              -3      0
4        5001712               0      C
...          ...             ...    ...
1048570  5150487             -25      C
1048571  5150487             -26      C
1048572  5150487             -27      C
1048573  5150487             -28      C
1048574  5150487             -29      C

[1048575 rows x 3 columns]


Let us see if the ID keys are all unique. If so, we want to change the index to correspond to the ID key. This will save time on later computations. 

In [323]:
print(len(set(app_df["ID"])))

438510


This is a bit unfortunate, as we have duplicate id's. We need to check if the duplicate ids have the same remaining data.

In [332]:
app_df_duplicate_id = app_df.duplicated(subset="ID", keep=False)

print(app_df[app_df_duplicate_id])

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
421211  7702516           F            N               Y             2   
421268  7602432           M            N               Y             0   
421349  7602432           F            N               N             0   
421464  7836971           M            Y               N             1   
421698  7213374           M            Y               N             0   
...         ...         ...          ...             ...           ...   
433158  7282535           F            N               Y             0   
433159  7742853           M            N               Y             0   
433217  7135270           F            N               Y             0   
433666  7091721           F            Y               Y             0   
433789  7618285           F            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
421211          180000.0      

There doesn't seem to be a particularly strong pattern between accounts tied to the same id. It is possible these are joint accounts but the demographic data doesn't really line up. We also don't have any indication from the dataset on how these id's are linked to the corresponding id in the credit dataframe. Fortuantely, we don't have many duplicates of this type, so we can just remove them from the dataset. 

In [474]:
app_df = app_df.drop_duplicates(subset="ID", keep=False)

Notice that we have yet another issue with duplicates: the same customers seem to have multiple account id's. We need to identify the cause of the duplication. It is possible these are multiple accounts for the same peron, and it is also possible these multiple accounts have different credit histories. We're going to collect all accounts with identical applicaton data together to then see how they behave in the credit. But before we do so, we will reduce both the application dataframe and the credit dataframe to just their intersections along the ID key. Additionaly, to make things easier, we will fill in values for null values in the application dataframe.

In [467]:
#Boolean series that tracks indices of app_df whose "ID" keys are also in credit_df

app_boolean = app_df["ID"].isin(credit_df["ID"])

app_df_overlap = app_df.loc[app_boolean]
print(app_df_overlap)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
1       5008805           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
4       5008809           F            N               Y             0   
...         ...         ...          ...             ...           ...   
434808  5149828           M            Y               Y             0   
434809  5149834           F            N               Y             0   
434810  5149838           F            N               Y             0   
434811  5150049           F            N               Y             0   
434812  5150337           M            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

Let us do the same for the credit data and only keep the overlap.

In [468]:
credit_boolean = credit_df["ID"].isin(app_df["ID"])

credit_df_overlap = credit_df.loc[credit_boolean]
print(credit_df_overlap)

              ID  MONTHS_BALANCE STATUS
92938    5008804               0      C
92939    5008804              -1      C
92940    5008804              -2      C
92941    5008804              -3      C
92942    5008804              -4      C
...          ...             ...    ...
1048570  5150487             -25      C
1048571  5150487             -26      C
1048572  5150487             -27      C
1048573  5150487             -28      C
1048574  5150487             -29      C

[777715 rows x 3 columns]


The account ID in the credit dataframe is not unique for each row. So, to check each account ID refers to a unique account, we will simply compare it to the number of account id's in the overlapping application dataframe. 

In [462]:
print(len(set(app_df_overlap["ID"])))
print(len(set(credit_df_overlap["ID"])))

36457
36457


This is excellent. It means each account is identified by its ID. We now collect all accounts with the same application data together, with the goal of creating a new dataframe that only has 1 row per customer, rather than 1 row per account, and whose ID column is replaced by a column that tracks all accounts for that customer. 

In [463]:
#Fill in missing values.

app_df_overlap = app_df_overlap.fillna(value="Null")

#Assigning variables to the columns of our app_df dataframe

columns = app_df.columns.values.tolist()
columns_remaining = [x for x in columns if x!='ID']

app_df_overlap_grouped = app_df_overlap.groupby(by=columns_remaining, axis=0)

In [475]:
#Defining a function that pulls out the id keys as a list from a dataframe corresponding to a group in the above groupby.

def IDList(df):
    '''Input is a dataframe with an ID column and other columns, where every row is identical except for having 
        different entries under ID.
       Returns a single row dataframe with all IDs collected into a single IDList column.'''
    list_of_ids = []
    
    for entry in df["ID"]:
        list_of_ids.append(entry)
    IDList_col = []
    for i in range(0, len(df.index.values)):
        IDList_col.append(list_of_ids)
        
    df["IDList"] = IDList_col
    return df



In [476]:
#Test
t_0 = time.time()
print(IDList(app_df_overlap.iloc[[0, 1], :]))
t_1 = time.time()
print("Time Elapsed: ", t_1 - t_0)
print(len(IDList(app_df_overlap.iloc[[0, 1], :])))

        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   

   AMT_INCOME_TOTAL NAME_INCOME_TYPE NAME_EDUCATION_TYPE NAME_FAMILY_STATUS  \
0          427500.0          Working    Higher education     Civil marriage   
1          427500.0          Working    Higher education     Civil marriage   

  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  FLAG_MOBIL  FLAG_WORK_PHONE  \
0  Rented apartment      -12005          -4542           1                1   
1  Rented apartment      -12005          -4542           1                1   

   FLAG_PHONE  FLAG_EMAIL OCCUPATION_TYPE  CNT_FAM_MEMBERS              IDList  
0           0           0            Null              2.0  [5008804, 5008805]  
1           0           0            Null              2.0  [5008804, 5008805]  
Time Elapsed:  0.0074841976165771484
2


<ipython-input-475-54fc612c9f13>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["IDList"] = IDList_col
<ipython-input-475-54fc612c9f13>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["IDList"] = IDList_col


In [479]:
app_df_id_list = app_df_overlap_grouped.apply(IDList)

In [480]:
print(app_df_id_list)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
1       5008805           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
4       5008809           F            N               Y             0   
...         ...         ...          ...             ...           ...   
434808  5149828           M            Y               Y             0   
434809  5149834           F            N               Y             0   
434810  5149838           F            N               Y             0   
434811  5150049           F            N               Y             0   
434812  5150337           M            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

We can now drop duplicates based on demographic data and we will have our desired dataframe. 

In [482]:
app_df_customer_rows = app_df_id_list.drop_duplicates(subset=columns_remaining)
print(app_df_customer_rows)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
7       5008812           F            N               Y             0   
10      5008815           M            Y               Y             0   
...         ...         ...          ...             ...           ...   
434797  5148694           F            N               N             0   
434801  5149055           F            N               Y             0   
434806  5149729           M            Y               Y             0   
434810  5149838           F            N               Y             0   
434812  5150337           M            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

We also create some functions that help us identify duplicate data. We first create a function that takes an index and returns all duplicates below it in the dataframe, assuming contiguous duplicates. We also create a function that returns the next index that isn't a duplicate of the given one. These will be important in constructing our label. To make the duplicate list function fast, we will want a quick way to conver ID's to indexes and indexes to ID's. This is necessary since our 

In [394]:
def DuplicateList(df, index, col_list):
    '''Parameters are a pandas dataframe, an index number and a list of column names. 
       Returns a list of all index values greater than or equal to the input index that have identical data under the given columns.
       Assumes the duplicated data in the dataframe is contiguous.'''
    
    Duplicates = [index]
    df_null = df.isnull()
    #Testing if the latest entry checked was a duplicate.
    is_Latest_Dupe = True
    for i in [x for x in df.index.values if x > index]:
        if is_Latest_Dupe:
            Dupe_Status = True
            for col in col_list:
                if df_null.loc[index, col] and df_null.loc[i, col]:
                    Dupe_Status=Dupe_Status
                else: 
                    Dupe_Status = Dupe_Status and (df.loc[index, col]==df.loc[i, col])
            if Dupe_Status:
                Duplicates.append(i)
            is_Latest_Dupe = is_Latest_Dupe and Dupe_Status
        elif not is_Latest_Dupe:
            break
    
    return Duplicates

Let's test this on some of the values we can see in our dataframe. 

In [395]:
print(DuplicateList(df=app_df, index=0 , col_list = columns_remaining))

[0, 1]


In [271]:
for i in [2, 3, 4, 438552, 438556]:
    print(DuplicateList(df=app_df, index=i, col_list=columns_remaining))
print(DuplicateList(app_df, index=1315, col_list=columns_remaining))
print(app_df.iloc[[1315, 1316],:])

[2]
[3, 4, 5, 6]
[4, 5, 6]
[438552]
[438556]
[1315, 1316, 1317]
           ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
1315  5010298           F            N               Y             0   
1316  5010299           F            N               Y             0   

      AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
1315          135000.0  Commercial associate  Secondary / secondary special   
1316          135000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
1315            Married  House / apartment      -14918          -1866   
1316            Married  House / apartment      -14918          -1866   

      FLAG_MOBIL  FLAG_WORK_PHONE  FLAG_PHONE  FLAG_EMAIL OCCUPATION_TYPE  \
1315           1                0           0           0     Sales staff   
1316           1                0           0           0     Sales staff   

      CNT_FAM_MEMBERS  
1315 

In [275]:
def NextNonDuplicate(df, index, col_list):
    '''Arguments are a dataframe with contiguous duplicate data, an index value and list of columns to compare
        for duplication.
       Returns the index of the next datapoint that isn\'t a duplicate of the data at index.'''
    if index <0:
        index = len(df.index.values)+index
    if index >=len(df.index.values)-1:
        return None
    elif DuplicateList(df, index, col_list)[-1] >= len(df.index.values)-1:
        return None
    else:
        return [DuplicateList(df, index, col_list)[-1]+1, df.loc[DuplicateList(df, index, col_list)[-1]+1, "ID"]]


We can now use the drop_duplicates method in pandas to remove duplicates from app_df, but this would create some issues. The account id's in the application data are linked to the account id's in the credit data. If we remove duplicates first, we will potentially lose information. Instead, we will adopt a different strategy. We identify the accounts that are in both dataframes and only condense each dataframe to those accounts for which we have both application data and credit data. This is a natural procedure to make here, as these are the only data points we can actually use to fit a model, as for other accounts, we either lack feature data or data to construct a label. 

In [312]:
def DuplicateIDs(df, ID, col_list):
    '''Parameters are a dataframe with an ID column and a specified ID number. 
       Returns a list of IDs that have the same data as the given ID across other columns'''
    t_0 = time.time()
    df_ID = df.loc[df["ID"]==ID]
    index = df_ID.index.values.tolist()[0]
    dupe_indices = DuplicateList(df, index, col_list)
    dupe_list = []
    for entry in df.loc[dupe_indices, "ID"]:
        dupe_list.append(entry)
    t_1 = time.time()
    print("Time Elapsed :", t_1 - t_0)
    return dupe_list

We're going to be using this function to construct our label. It will be applied many times, so we want to get a sense of how long this function takes. 

In [318]:
print(DuplicateIDs(app_df, 5008804, columns_remaining))


print(DuplicateIDs(app_df, 5008808, columns_remaining))


0.1725151538848877
Time Elapsed : 0.1752791404724121
[5008804, 5008805]
0.1635420322418213
Time Elapsed : 0.16531777381896973
[5008808, 5008809, 5008810, 5008811]


Let's now write a function that will take our credit dataframe and extract the entries whose id numbers correspond to duplicates of a given id.

In [259]:
def GetDuplicateData(ID, output_df=credit_df_only_including_overlap, identifier_df=app_df_only_including_overlap):
    '''Parameters: ID is an id number. output_df and identifier_df are dataframes that have an ID column 
        with all ID values shared in common in both dataframes.
       Returns: Entries in output_df that correspond to the duplicate id_values, with duplicates identified 
        via identifier_df'''
    id_series=DuplicateIDs(identifier_df, ID)
    
    return output_df.loc[output_df["ID"].isin(id_series)]
    

Here are some examples. 

In [262]:
print(GetDuplicateData(5008804))

            ID  MONTHS_BALANCE STATUS
92938  5008804               0      C
92939  5008804              -1      C
92940  5008804              -2      C
92941  5008804              -3      C
92942  5008804              -4      C
92943  5008804              -5      C
92944  5008804              -6      C
92945  5008804              -7      C
92946  5008804              -8      C
92947  5008804              -9      C
92948  5008804             -10      C
92949  5008804             -11      C
92950  5008804             -12      C
92951  5008804             -13      1
92952  5008804             -14      0
92953  5008804             -15      X
92954  5008805               0      C
92955  5008805              -1      C
92956  5008805              -2      C
92957  5008805              -3      C
92958  5008805              -4      C
92959  5008805              -5      C
92960  5008805              -6      C
92961  5008805              -7      C
92962  5008805              -8      C
92963  50088

In [284]:
print(GetDuplicateData(app_df_only_including_overlap.loc[3, "ID"]).iloc[0:40])

            ID  MONTHS_BALANCE STATUS
92999  5008808               0      0
93000  5008808              -1      X
93001  5008808              -2      0
93002  5008808              -3      X
93003  5008808              -4      X
93004  5008809             -22      X
93005  5008809             -23      X
93006  5008809             -24      X
93007  5008809             -25      X
93008  5008809             -26      X
93009  5008810               0      C
93010  5008810              -1      C
93011  5008810              -2      C
93012  5008810              -3      C
93013  5008810              -4      C
93014  5008810              -5      C
93015  5008810              -6      C
93016  5008810              -7      C
93017  5008810              -8      C
93018  5008810              -9      C
93019  5008810             -10      C
93020  5008810             -11      C
93021  5008810             -12      C
93022  5008810             -13      C
93023  5008810             -14      C
93024  50088

In [285]:
print(GetDuplicateData(app_df_only_including_overlap.loc[3, "ID"]).iloc[40:])

            ID  MONTHS_BALANCE STATUS
93039  5008811              -3      C
93040  5008811              -4      C
93041  5008811              -5      C
93042  5008811              -6      C
93043  5008811              -7      C
93044  5008811              -8      C
93045  5008811              -9      C
93046  5008811             -10      C
93047  5008811             -11      C
93048  5008811             -12      C
93049  5008811             -13      C
93050  5008811             -14      C
93051  5008811             -15      C
93052  5008811             -16      C
93053  5008811             -17      C
93054  5008811             -18      C
93055  5008811             -19      C
93056  5008811             -20      C
93057  5008811             -21      C
93058  5008811             -22      C
93059  5008811             -23      C
93060  5008811             -24      C
93061  5008811             -25      C
93062  5008811             -26      C
93063  5008811             -27      X
93064  50088

In [290]:
print(NextNonDuplicate(app_df, 3, columns_remaining))
print(DuplicateList(app_df, 7, columns_remaining))

[7, 5008812]
[7, 8, 9]


In [291]:
print(GetDuplicateData(app_df_only_including_overlap.loc[7, "ID"]))

            ID  MONTHS_BALANCE STATUS
93075  5008812              -4      0
93076  5008812              -5      X
93077  5008812              -6      X
93078  5008812              -7      0
93079  5008812              -8      0
93080  5008812              -9      0
93081  5008812             -10      0
93082  5008812             -11      0
93083  5008812             -12      0
93084  5008812             -13      0
93085  5008812             -14      0
93086  5008812             -15      0
93087  5008812             -16      0
93088  5008812             -17      0
93089  5008812             -18      0
93090  5008812             -19      0
93091  5008812             -20      X
93092  5008813               0      0
93093  5008813              -1      X
93094  5008813              -2      X
93095  5008813              -3      0
93096  5008813              -4      0
93097  5008813              -5      0
93098  5008813              -6      0
93099  5008813              -7      0
93100  50088

These tables are suggesgtive. The different account ids in application data are linked to the same person, but correspond to different credit accounts with the bank the data was collected from. This suggests an approach to deal with the duplication:
1. Use drop_duplicate to only keep the top entry in each block corresponding to a single person.
2. Record the other account numbers of the person as a single new column in the application dataframe.
3. Use the credit data for all accounts of this person to create a new label column in the application dataframe. 

Some of this may involve a lot of processing time. As a quick check on whether we can save some effort, let us see how much data we lose if we simply keep just one account per customer. This will also give us the number of customers we have data for. 

In [294]:
app_no_dupes = app_df_only_including_overlap.drop_duplicates(subset=columns_remaining)

In [295]:
print(app_no_dupes)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
7       5008812           F            N               Y             0   
10      5008815           M            Y               Y             0   
...         ...         ...          ...             ...           ...   
434797  5148694           F            N               N             0   
434801  5149055           F            N               Y             0   
434806  5149729           M            Y               Y             0   
434810  5149838           F            N               Y             0   
434812  5150337           M            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

We have roughly a 4:1 ratio of accounts:customers, so it will be important to track multiple accounts for the same person. To keep track of the data, we add the duplicate id's as a column to the dataframe without duplicate accounts.